In [ ]:
import os
import pandas as pd

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import bnlearn as bn

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
from utils import read_data
combined_data = read_data()

In [ ]:
select_group = [

    # 號誌
    '號誌-號誌種類名稱', 
    # '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    # '車輛撞擊部位大類別名稱-最初',
    # '事故類型及型態大類別名稱', '車道劃分設施-分向設施大類別名稱',
    # '道路型態大類別名稱',
    # '車輛撞擊部位子類別名稱-最初',
    '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    '道路型態子類別名稱',

    # 其他
    '速限-第1當事者',
    '道路類別-第1當事者-名稱',

    # 設施
    'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',
    
    # 駕駛、行人行為
    '肇因研判子類別名稱-主要'
    ]

In [ ]:
data = combined_data[select_group].copy()
data['facility'] = data[['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count']].apply(
    lambda row: '1' if (row > 0).any() else '0', axis=1
)
data.drop(columns=['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count'], inplace=True)

max_speed = data['速限-第1當事者'].max()
bins = range(0, int(max_speed) + 11, 10)

data['速限-第1當事者'] = pd.cut(
    data['速限-第1當事者'],
    bins=bins,
    right=False, 
    include_lowest=True,
    labels=[f"{i}-{i+9}" for i in bins[:-1]]
)

In [ ]:
for i in data.columns:
    print(i, len(data[i].unique()))

In [ ]:
# 學哪些變數之間有邊，結果是一個DAG
model = bn.structure_learning.fit(data, methodtype='hc', scoretype='bic', bw_list_method='edges',
                                  # 肇因對於事故類型一定是上游
                                   black_list=[('事故類型及型態子類別名稱', '肇因研判子類別名稱-主要')])
# 計算每個節點的 條件機率表 (CPT, Conditional Probability Table)
model_param = bn.parameter_learning.fit(model, data)
# 計算邊緣強度，如果p小於顯著就是有相關
model_independence = bn.independence_test(model_param, data, test='chi_square', prune=True)

In [ ]:
model_independence['independence_test']

In [ ]:
bn.get_parents(model['model_edges'])

In [ ]:
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei']
rcParams['axes.unicode_minus'] = False

# G = bn.plot(model, interactive=False, node_color="#36AA5B", edge_labels=None)
bn.plot(model_independence, interactive=False, edge_labels='pvalue', 
        params_static={'layout': 'spring_layout', 'font_color': "#974848", 'edge_color': "#974848"})

In [ ]:
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from utils_behaviour import feature_name_map

def draw_bn_plotly(model, alpha=0.05, layout_algo="", en=False):
    edges = [(str(u), str(v)) for u, v in model['model_edges']]
    df = model['independence_test'][['source','target','p_value']].copy()

    if en:
        df['source'] = df['source'].map(feature_name_map).fillna(df['source'])
        df['target'] = df['target'].map(feature_name_map).fillna(df['target'])
        edges = [(feature_name_map.get(u, u), feature_name_map.get(v, v)) for (u, v) in edges]
    else:
        df['source'] = df['source'].astype(str)
        df['target'] = df['target'].astype(str)

    p_map = {(s,t):p for s,t,p in df.itertuples(index=False, name=None)}
    p_map.update({(t,s):p for (s,t),p in list(p_map.items())})

    G = nx.DiGraph()
    G.add_edges_from(edges)

    pos = (nx.spring_layout(G, seed=42) if layout_algo=="spring"
           else nx.kamada_kawai_layout(G))

    # nodes
    deg = dict(G.degree())
    node_x, node_y, node_text, node_size = [], [], [], []
    for n in G.nodes():
        x,y = pos[n]
        node_x.append(x); node_y.append(y)
        node_text.append(f"{n}<br>degree: {deg.get(n,0)}")
        node_size.append(10 + 25*(deg.get(n,1)))

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=[str(n) for n in G.nodes()], textposition="top center",
        hovertext=node_text, hoverinfo="text",
        marker=dict(size=node_size, line=dict(width=1), color="#63B1E5")
    )

    # edges
    edge_traces = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        p = p_map.get((u,v), np.nan)
        if np.isnan(p):
            width, dash, color = 1.0, "dot", "#999"
            tip = f"{u} → {v}<br>p-value: N/A"
        else:
            w = -np.log10(max(p, 1e-300))
            width = 1 # 1 + 0.8*min(10, w)
            sig = (p <= alpha)
            dash = "solid" if sig else "dot"
            color = "#7dddf7" if sig else "#78c6fd"
            tip = f"{u} → {v}<br>p-value: {p:.3e}"

        edge_traces.append(go.Scatter(
            x=[x0, x1], y=[y0, y1],
            mode='lines',
            hoverinfo='text', text=[tip],
            line=dict(width=width, color=color, dash=dash)
        ))


    # 箭頭
    annotations = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        annotations.append(dict(
            ax=x0, ay=y0, x=x1, y=y1,
            xref="x", yref="y", axref="x", ayref="y",
            showarrow=True, arrowhead=3, arrowsize=2, opacity=0.8
        ))

    fig = go.Figure(data=edge_traces + [node_trace],
        layout=go.Layout(
            template=None, showlegend=False,
            hovermode='closest',
            margin=dict(l=10, r=10, t=10, b=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            annotations=annotations,
            width=1300, height=600,
        )
    )
    return fig

draw_bn_plotly(model_independence, alpha=0.01, layout_algo='', en=True)

In [ ]:
# Conditional Probability Distributions (CPDs)
CPDs = bn.print_CPD(model_param)

In [ ]:
# # ## 肇因研判依賴於號誌種類
# parent  = '號誌-號誌種類名稱'
# child = '肇因研判子類別名稱-主要'

# ## 事故類型依賴於肇因研判
# parent = '肇因研判子類別名稱-主要'
# child  = '事故類型及型態子類別名稱'

# ## 快車道、一般車道依賴於肇因研判
# parent = '肇因研判子類別名稱-主要'
# child  = '車道劃分設施-分道設施-快車道或一般車道間名稱'

## 道路型態依賴於肇因研判、號誌種類
parent = '肇因研判子類別名稱-主要'
parent2 = '號誌-號誌種類名稱'
child  = '道路型態子類別名稱'


In [ ]:
from utils_behaviour import category_value_map

dfprob_cause = CPDs[child].sort_values('p', ascending=False)

counts = (data.groupby([child, parent, parent2]).size().reset_index(name='n'))
# counts = (data.groupby([child, parent]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent, parent2], how='left')
    # .merge(counts, on=[child, parent], how='left')
    .sort_values('p', ascending=False)
)

filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30].head(30)
filtered['p'] = round(filtered['p'], 4)

filtered[parent] = filtered[parent].map(category_value_map[parent])
filtered[child] = filtered[child].map(category_value_map[child])
filtered[parent2] = filtered[parent2].map(category_value_map[parent2])
filtered

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from utils_behaviour import BubbleChart

# filtered['特徵組合'] = filtered[parent] + '\n->' + filtered[child]
filtered['特徵組合'] = filtered[parent] + '+\n' + filtered[parent] + '\n->' + filtered[child]

labels = filtered['特徵組合'].reset_index(drop=True)
n_values =  filtered['n'].reset_index(drop=True)

cmap = cm.Blues
norm = mcolors.Normalize(vmin=n_values.min(), vmax=n_values.max())
colors = cmap(norm(n_values.values))

bubble_chart = BubbleChart(area=filtered['p'],
                           bubble_spacing=0.1, text_rotation=20)
bubble_chart.collapse()

fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"), figsize=(18,10))
bubble_chart.plot(ax, labels, colors)
for t in ax.texts:
    t.set_fontsize(8)

ax.axis("off")
ax.relim()
ax.autoscale_view()
ax.set_title('Bubble Chart')

sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
fig.colorbar(sm, ax=ax, label='n value')

plt.show()
